In [1]:
import torch
import torch.nn as nn
import numpy as np
from tqdm import tqdm
from torch.cuda.amp import autocast
import cv2
import torch
import torch.nn.functional as F
from math import ceil
import os, sys
from glob import glob
import matplotlib.pyplot as plt
import pandas as pd
import segmentation_models_pytorch as smp
from torch.utils.data import Dataset, DataLoader
from torch.nn.parallel import DataParallel
from PIL import Image
from dotenv import load_dotenv

In [2]:
class CFG:
    # ============== model CFG =============
    model_name = "Unet"
    backbone = "se_resnext50_32x4d"

    in_chans = 5  # 65
    # ============== _ CFG =============
    image_size = 512
    input_size = 512
    tile_size = image_size
    stride = tile_size // 4
    drop_egde_pixel = 32

    target_size = 1
    chopping_percentile = 1e-3
    # ============== fold =============
    valid_id = 1
    batch = 128
    th_percentile = 0.0021
    model_path = [
        "/kaggle/input/sennet-yoyobar-v6-model/se_resnext50_32x4d_19_loss0.12_score0.79_val_loss0.25_val_score0.79.pt"
    ]

In [3]:
class CustomModel(nn.Module):
    def __init__(self, CFG, weight=None):
        super().__init__()
        self.CFG = CFG
        self.model = smp.Unet(
            encoder_name=CFG.backbone,
            encoder_weights=weight,
            in_channels=CFG.in_chans,
            classes=CFG.target_size,
            activation=None,
        )
        self.batch = CFG.batch

    def forward(self, image):
        output = self.model(image)

        return output


net = CustomModel(CFG).cuda()
net.load_state_dict(torch.load(CFG.model_path[0], "cpu"))
net.eval();

In [4]:
def min_max_normalization(x: torch.Tensor) -> torch.Tensor:
    """input.shape=(batch,f1,...)"""
    shape = x.shape
    if x.ndim > 2:
        x = x.reshape(x.shape[0], -1)

    min_ = x.min(dim=-1, keepdim=True)[0]
    max_ = x.max(dim=-1, keepdim=True)[0]
    if min_.mean() == 0 and max_.mean() == 1:
        return x.reshape(shape)

    x = (x - min_) / (max_ - min_ + 1e-9)
    return x.reshape(shape)


def norm_with_clip(x: torch.Tensor, smooth=1e-5):
    dim = list(range(1, x.ndim))
    mean = x.mean(dim=dim, keepdim=True)
    std = x.std(dim=dim, keepdim=True)
    x = (x - mean) / (std + smooth)
    x[x > 5] = (x[x > 5] - 5) * 1e-3 + 5
    x[x < -3] = (x[x < -3] + 3) * 1e-3 - 3
    return x


class Data_loader(Dataset):
    def __init__(self, path, s="/images/"):
        self.paths = glob(path + f"{s}*.tif")
        self.paths.sort()
        self.bool = s == "/labels/"

    def __len__(self):
        return len(self.paths)

    def __getitem__(self, index):
        img = cv2.imread(self.paths[index], cv2.IMREAD_GRAYSCALE)
        img = torch.from_numpy(img)
        if self.bool:
            img = img.to(torch.bool)
        else:
            img = img.to(torch.uint8)
        return img


def load_data(path, s):
    data_loader = Data_loader(path, s)
    data_loader = DataLoader(data_loader, batch_size=16, num_workers=2)
    data = []
    for x in tqdm(data_loader):
        data.append(x)
    x = torch.cat(data, dim=0)
    ########################################################################
    TH = x.reshape(-1).numpy()
    index = -int(len(TH) * CFG.chopping_percentile)
    TH: int = np.partition(TH, index)[index]
    x[x > TH] = int(TH)
    ########################################################################
    TH = x.reshape(-1).numpy()
    index = -int(len(TH) * CFG.chopping_percentile)
    TH: int = np.partition(TH, -index)[-index]
    x[x < TH] = int(TH)
    ########################################################################
    # x=(min_max_normalization(x.to(torch.float16))*255).to(torch.uint8)
    return x


class Pipeline_Dataset(Dataset):
    def __init__(self, x, path):
        self.img_paths = glob(path + "/images/*")
        self.img_paths.sort()
        self.in_chan = CFG.in_chans
        z = torch.zeros(self.in_chan // 2, *x.shape[1:], dtype=x.dtype)
        self.x = torch.cat((z, x, z), dim=0)
        self.msks_dir = f"/kaggle/input/blood-vessel-segmentation/train/kidney_1_dense/labels/"

    def __len__(self):
        return self.x.shape[0] - self.in_chan + 1

    def __getitem__(self, index):

        x = self.x[index : index + self.in_chan]
        msk_path = self.msks_dir + f"{index:04}.tif"
        msk = np.array(Image.open(msk_path))
        msk = torch.as_tensor(msk, dtype=torch.float32)
        msk /= 255

        return x, msk, index

In [5]:
class Patcher:
    def __init__(self, h, w, patch_size, stride):
        self.h = h
        self.w = w
        self.patch_size = patch_size
        self.stride = stride
        self.h_pad = self.stride * ceil((h - patch_size) / self.stride) + patch_size - h
        self.w_pad = self.stride * ceil((w - patch_size) / self.stride) + patch_size - w

        self.unfold = torch.nn.Unfold(kernel_size=(patch_size, patch_size), stride=self.stride)
        self.fold = torch.nn.Fold(
            output_size=(h + self.h_pad, w + self.w_pad), kernel_size=(patch_size, patch_size), stride=self.stride
        )

    def extract_patches(self, x):
        assert x.ndim == 4
        x = x.float()
        bs, c, _, _ = x.shape

        x = F.pad(x, (0, self.w_pad, 0, self.h_pad), mode="reflect")
        patches = self.unfold(x).view(bs, 5, self.patch_size, self.patch_size, -1).permute(0, 4, 1, 2, 3)
        return patches

    def merge_patches(self, patches):
        bs = len(patches)
        average_mask = 1 / self.fold(
            self.unfold(torch.ones(1, self.h + self.h_pad, self.w + self.w_pad, device=patches.device))
        )

        x_restored = self.fold(patches.permute(0, 2, 3, 1).view(bs, self.patch_size * self.patch_size, -1))
        x_restored = x_restored * average_mask
        x_restored = x_restored[:, :, : self.h, : self.w]

        return x_restored

In [6]:
path = "/kaggle/input/blood-vessel-segmentation/train/kidney_1_dense"
bs = 2

x = load_data(path, "/images/")
dataset = Pipeline_Dataset(x, path)
dataloader = DataLoader(dataset, batch_size=bs, shuffle=False, num_workers=2)

h, w = x.shape[-2:]
patcher = Patcher(h, w, CFG.tile_size, CFG.stride)
kidney_probs = torch.zeros_like(x, dtype=torch.float32)

100%|██████████| 143/143 [00:07<00:00, 19.79it/s]


In [7]:
# img, target, idx = next(iter(dataloader))
for img, target, idx in tqdm(dataloader):
    img = img.float().cuda()  # (bs, 5, 1303, 912)
    target = target.cuda()
    img = norm_with_clip(img.reshape(-1, *x.shape[2:])).reshape(img.shape)

    patches = patcher.extract_patches(img)  # (bs, n_patches, 5, 512, 512)
    patches = patches.reshape(-1, 5, CFG.tile_size, CFG.tile_size)

    with torch.no_grad():
        probs = net(patches).sigmoid()  # (bs * n_patches, 1, 512, 512)

    probs = probs.reshape(bs, -1, CFG.tile_size, CFG.tile_size)
    probs = patcher.merge_patches(probs).squeeze()  # (bs, 1303, 912)
    kidney_probs[idx] = probs.cpu()

100%|█████████▉| 1139/1140 [33:48<00:01,  1.78s/it]


RuntimeError: Given output_size=(1408, 1024), kernel_size=(512, 512), dilation=(1, 1), padding=(0, 0), stride=(128, 128), expected size of input's dimension 2 to match the calculated number of sliding blocks 8 * 5 = 40, but got input.size(2)=20.

In [ ]:
torch.save(kidney_probs, 'kidney_probs.pt')